**Experiment 2: DTA, linear regression and selecting features that occur a certain amount of time**

*Background*: Experiment 1 showed that some sort of feature selection is required, but also that just picking the k highest scoring features leads to overfitting.

*Goal*: Determine if it is possible to predict the year in which a text was written using regression.

*Strategies*:
- Train on features that occur a certain amount of time

*Relevance*:
- If this experiment works, it is possible to estimate years for corpora that have NA's in this variable.

*Success criteria*:
- Consistent findings over training-, test- and validation set
- predicted year is not more than ten years away from the true year

*Corpora*:
- DTA

*Result*: 
Only using features that occur in 800 out of 899 documents solved the problem of overfitting.

The linear regression seems to try to model a normal distribution, which does not reflect the real distribution of years in the DTA (cf analysis notebook). Therefore, it seems that linear regression is not flexible enough to model the data correctly.

In [36]:
import pandas as pd
import numpy as np
import nltk

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest , f_regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import sklearn.utils
import re

import eli5

In [48]:
#Starting R for statistical analysis
from rpy2 import robjects

from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

from rpy2.robjects.packages import importr



R = robjects.r

ggplot = importr('ggplot2')

tidyr = importr('tidyr')

dplyr = importr('dplyr')

In [2]:
# Code example: https://stackoverflow.com/questions/39839112/the-easiest-way-for-getting-feature-names-after-running-selectkbest-in-scikit-le
def features_to_names(features, feature_names):
    features_selected = []

    for bool, feature in zip(features, feature_names):
        if bool:
            features_selected.append(feature)
    return features_selected

In [3]:
train_full = pd.read_csv('/Volumes/Korpora/Train/DTA_train_tokenized.csv', sep=';')
val_full = pd.read_csv('/Volumes/Korpora/Val/DTA_val_tokenized.csv', sep=';')
test_full = pd.read_csv('/Volumes/Korpora/Test/DTA_test_tokenized.csv', sep=';')

In [4]:
print('Length train set: ',len(train_full))
print('Length validation set: ', len(val_full))
print('Length test set: ', len(test_full))

Length train set:  899
Length validation set:  225
Length test set:  281


In [5]:
#build tokenizer that just substitutes '[' and ']' with ','
def tokenizer_word(doc):
    doc = re.sub('[(\[+)|(\]+)]', '', doc)
    doc = re.split(',', doc)
    return doc

In [88]:
#function for assembling predictions in order to find out how features are weighted

def collect_predictions(dataset, classifier,vectorizer, feature_names, pipeline):
    predictions = eli5.explain_weights_df(classifier,vec=vectorizer, feature_names=feature_names)
    
    predictions = predictions.drop(['target'], axis=1)
    
    
    predictions['YEAR'] = 0
    
    

    for instance in range (0, len(dataset)):
        pred = eli5.explain_prediction_df(classifier, dataset[instance], vec=vectorizer, feature_names=feature_names)
        source_text = pd.DataFrame([[dataset[instance]]])
        year_pred = pipeline.predict(source_text[0])
        pred['weight_value'] = pred['weight'] * pred['value']
        pred['instance'] = instance
        
        
        pred = pred.drop(['target','weight','value'], axis=1)
        
    
        pred['YEAR'] = np.round(year_pred[0])
    
        predictions = pd.concat([predictions, pred])
    
    
    
    return predictions

In [7]:
train_x = train_full['Text']
train_y = train_full['Publication_year']

val_x = val_full['Text']
val_y = val_full['Publication_year']

CountVectorizer has an attribute called min_df that can be set to an integer or a float. If a word occurs less often than min_df (count or distribution), it is removed. I set min_df to 10 for the next experiment.

In Experiment 1, the model performed best on a set with 22000 features, so I start with that value.

In [ ]:
reg_1 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 10)),
                    ('feature_selector', SelectKBest(f_regression, k=22000)),
                         ('ridge_reg', linear_model.Ridge())
                        ])



In [ ]:
reg_1.fit(train_x, train_y)

In [ ]:
y_pred_train = reg_1.predict(train_x)
mean_squared_error(train_y, y_pred_train)

In [ ]:
y_pred_val = reg_1.predict(val_x)

mean_squared_error(val_y, y_pred_val)

In [ ]:
features = reg_1['feature_selector'].get_support()
feature_names = reg_1['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

expl = eli5.explain_weights_df(reg_1['ridge_reg'],vec=reg_1['unigram_vectorizer'],target_names=(train_y),feature_names=features_selected)




In [ ]:
print(expl.head(2))

It is still overfitting heavily, so I raise the threshold to 100.

Since there are less than 22k features over the threshold, I set k to 'all' for now, because else the feature selector does complain.

In [ ]:
reg_2 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 100)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [ ]:
reg_2.fit(train_x, train_y)

In [ ]:
y_pred_train = reg_2.predict(train_x)
mean_squared_error(train_y, y_pred_train)

In [ ]:
y_pred_val = reg_2.predict(val_x)

mean_squared_error(val_y, y_pred_val)

In [ ]:
features = reg_2['feature_selector'].get_support()
feature_names = reg_2['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.show_weights(reg_2['ridge_reg'],vec=reg_2['unigram_vectorizer'], feature_names=features_selected)

In [ ]:
reg_3 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 200)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [ ]:
reg_3.fit(train_x, train_y)

In [ ]:
y_pred_train = reg_3.predict(train_x)
mean_squared_error(train_y, y_pred_train)

In [ ]:
y_pred_val = reg_3.predict(val_x)

mean_squared_error(val_y, y_pred_val)

In [ ]:
features = reg_3['feature_selector'].get_support()
feature_names = reg_3['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.show_weights(reg_3['ridge_reg'],vec=reg_3['unigram_vectorizer'], feature_names=features_selected)

Since the model is still overfitting, I reverse the experiment, and I just pick features that occur in 800 out of 899 documents.

In [8]:
reg_4 = Pipeline([ ('unigram_vectorizer', CountVectorizer(tokenizer=tokenizer_word, min_df = 800)),
                    ('feature_selector', SelectKBest(f_regression, k='all')),
                         ('ridge_reg', linear_model.Ridge())
                        ])

In [9]:
reg_4.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('unigram_vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=800,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_word at 0x1ddc45c20>,
                                 vocabulary=None)),
                ('feature_selector',
                 SelectKBest(k='all',
                             score_func=<function f_regression at 0x125002cb0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=Tr

In [10]:
y_pred_train = reg_4.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2259.798810884709

In [11]:
y_pred_val = reg_4.predict(val_x)

mean_squared_error(val_y, y_pred_val)

3202.5089784523752

In [12]:
features = reg_4['feature_selector'].get_support()
feature_names = reg_4['unigram_vectorizer'].get_feature_names()

features_selected = features_to_names(features, feature_names)

eli5.explain_weights(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

Explanation(estimator="Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,\n      normalize=False, random_state=None, solver='auto', tol=0.001)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1769.72498378178, std=None, value=None), FeatureWeight(feature=" 'keine'", weight=0.28215865189276, std=None, value=None), FeatureWeight(feature=" 'weiter'", weight=0.2605753331467955, std=None, value=None), FeatureWeight(feature=" 'ja'", weight=0.21697707009698006, std=None, value=None), FeatureWeight(feature=" 'that'", weight=0.18137675105525447, std=None, value=None), FeatureWeight(feature=" 'leicht'", weight=0.17787227899112906, std=None, value=None), FeatureWeight(feature=" 'vielen'", weight=0.17585700726028217, std=None, value=None), FeatureWeight(feature=" 'finden'", weight=0.15523597279447018, std=None, value=None), FeatureWeight(feature=" 'erhalten'", weight=0.1531355142374395, std=None, value=None), FeatureWeight(feature=" 'liegen'", weight=0.1520930530436624, std=None, value=None)], neg=[FeatureWeight(feature=" 'kein'", weight=-0.3395522074968795, std=None, value=None), FeatureWeight(feature=" 'weit'", weight=-0.22985734790484544, std=None, value=None), FeatureWeight(feature=" 'ins'", weight=-0.22189071325635346, std=None, value=None), FeatureWeight(feature=" 'bald'", weight=-0.1796849913742998, std=None, value=None), FeatureWeight(feature=" 'gen'", weight=-0.17659622069132203, std=None, value=None), FeatureWeight(feature=" 'genug'", weight=-0.17566194219113015, std=None, value=None), FeatureWeight(feature=" 'einander'", weight=-0.17344353728272038, std=None, value=None), FeatureWeight(feature=" 'alle'", weight=-0.16552632871642325, std=None, value=None), FeatureWeight(feature=" 'welchen'", weight=-0.1647060671157467, std=None, value=None), FeatureWeight(feature=" 'jeden'", weight=-0.15880460654825063, std=None, value=None)], pos_remaining=110, neg_remaining=86), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [13]:
len(features_selected)

215

The MSE of the train and val set converged now, so going after the stop words seems to be a pretty good idea. The model trained on 215 features.

In [14]:
y_pred_train = pd.DataFrame(y_pred_train, columns=['Predicted_y'])

diff_pred_true_train = pd.concat([y_pred_train, train_y], axis=1)

diff_pred_true_train['Difference'] = diff_pred_true_train.Predicted_y - diff_pred_true_train.Publication_year
    

print(diff_pred_true_train.head(3))


   Predicted_y  Publication_year  Difference
0  1786.259536              1741   45.259536
1  1750.030794              1691   59.030794
2  1770.757624              1665  105.757624


In [15]:
diff_pred_true_train.describe()

Predicted_y  Publication_year    Difference
count   899.000000        899.000000  8.990000e+02
mean   1788.259177       1788.259177 -4.426073e-14
std      61.741568         77.929074  4.756380e+01
min    1549.902463       1598.000000 -1.258733e+02
25%    1762.839429       1739.500000 -2.805162e+01
50%    1782.950146       1796.000000  2.362674e-02
75%    1824.280243       1855.000000  2.514667e+01
max    1962.413722       1913.000000  1.671227e+02

This table shows that the mean of the years is the same for the predicted and the true year. The maximum and minimum of the model's prediction is lower and higher than in the true labels, so the model thinks that the range between the earliest and the latest publication year is larger than shown in the train set.

In [16]:
y_pred_val = pd.DataFrame(y_pred_val, columns=['Predicted_y'])

diff_pred_true_val = pd.concat([y_pred_val, val_y], axis=1)

diff_pred_true_val['Difference'] = diff_pred_true_val.Predicted_y - diff_pred_true_val.Publication_year

print(diff_pred_true_val.head(3))


   Predicted_y  Publication_year  Difference
0  1818.768364              1897  -78.231636
1  1717.750097              1701   16.750097
2  1517.937050              1663 -145.062950


In [17]:
diff_pred_true_val.describe()

Predicted_y  Publication_year  Difference
count   225.000000        225.000000  225.000000
mean   1789.651089       1791.315556   -1.664467
std      70.223421         74.822785   56.692355
min    1482.561566       1603.000000 -164.438434
25%    1763.628141       1750.000000  -35.216816
50%    1784.658281       1804.000000   -5.313875
75%    1822.392752       1843.000000   28.570888
max    2025.898784       1913.000000  255.349277

The true mean of the publication year in the validation set is three years higher than in the train set. The model adapts slightly by adding one year to the mean of the predictions over the validation set (compared to the train set).

Surprisingly, the model dates the oldest text from the validation set back to 1462, when in fact, the oldest text was written in 1603. The youngest text in the validation set, according to the model, was written in 2014, the true year of the youngest text is 1913. This means that the range of the prediction is about 350 years larger than it should be.

The mean difference between the predicted and the true year is -2, meaning that the predicted year is generally two years lower than the true label.

in the first quartile, the models prediction is about 37 years to low, in the third quartile, the prediction is about 256 years to high. It seems that the model generally tends to predict a higher publication year than the true year. Given the mean (which is actually quite decent), the main problem might be some heavy outliers in the model's prediction.

In [18]:
print(diff_pred_true_val.nsmallest(10,'Difference'))

     Predicted_y  Publication_year  Difference
143  1482.561566              1647 -164.438434
183  1531.163409              1679 -147.836591
2    1517.937050              1663 -145.062950
170  1763.628141              1895 -131.371859
54   1775.478628              1897 -121.521372
188  1779.429106              1893 -113.570894
159  1783.201390              1890 -106.798610
50   1784.179079              1889 -104.820921
47   1801.314523              1898  -96.685477
116  1820.343756              1913  -92.656244


eli5 instance 119 and 142: The model predicts 2012 and 2014 (true: 1765 and 1895), probably because it overestimates the influence of the word "dem". In the instance of 2012, "dem" has a weight of +348, whereas for the one with 2014, it is weighted with +71. 

"die" seems also to be a word that misleads the classifier to think that a text is younger than it really is. For the text of 2012, "die" has a weight of +185, for the example that was predicted with 2014, the weight is +65

In [95]:
diff_pred_true_val.to_csv('/Volumes/Korpora/Reg4_Labels_val.csv',sep=';')

In [96]:
diff_pred_true_train.to_csv('/Volumes/Korpora/Reg4_Labels_train.csv',sep=';')

In [19]:
#eli5.explain_prediction(reg_4['ridge_reg'],val_x[143],vec=reg_4['unigram_vectorizer'], feature_names=features_selected, )

In [20]:
expl = eli5.explain_prediction_df(reg_4['ridge_reg'],val_x[0],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)



In [21]:
len(expl)

213

In [22]:
expl.nsmallest(10,'value')

target      feature       weight  value
0        y       <BIAS>  1769.724984    1.0
111      y    'welchem'     0.112434    1.0
113      y   'gleichen'     0.080908    1.0
100      y     'findet'     0.240282    2.0
116      y    'welches'     0.016883    2.0
117      y        'nen'     0.014924    2.0
106      y     'bleibt'     0.174982    3.0
127      y     'zeiten'    -0.310498    3.0
83       y     'liegen'     0.608372    4.0
112      y      'dahin'     0.083053    4.0

In [23]:
eli5.explain_weights_df(reg_4['ridge_reg'],vec=reg_4['unigram_vectorizer'], feature_names=features_selected)

target    feature       weight
0        y     <BIAS>  1769.724984
1        y    'keine'     0.282159
2        y   'weiter'     0.260575
3        y       'ja'     0.216977
4        y     'that'     0.181377
..     ...        ...          ...
211      y      'gen'    -0.176596
212      y     'bald'    -0.179685
213      y      'ins'    -0.221891
214      y     'weit'    -0.229857
215      y     'kein'    -0.339552

[216 rows x 3 columns]

In [89]:
val_details = collect_predictions(val_x, reg_4['ridge_reg'],reg_4['unigram_vectorizer'],features_selected, reg_4)



In [91]:
len(val_details)

46786

In [93]:
train_details = collect_predictions(train_x, reg_4['ridge_reg'],reg_4['unigram_vectorizer'],features_selected, reg_4)

In [94]:
train_details.to_csv('/Volumes/Korpora/Exp2_Reg4_Train_results.csv',sep=';')
val_details.to_csv('/Volumes/Korpora/Exp2_Reg4_Val_results.csv', sep=';')

R object with classes: ('data.frame',) mapped to:
<DataFrame - Python:0x1f1fba4b0 / R:0x7f957b217600>
[IntVector, FloatVector, FloatVector, FloatVector, ..., FloatVector, FloatVector, FloatVector, FloatVector]
  instance: <class 'rpy2.robjects.vectors.IntVector'>
  R object with classes: ('integer',) mapped to:
<IntVector - Python:0x15b4025f0 / R:0x7f957fe4e8b8>
[0, 1, 2, 3, 4, 5]
  ': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x15b402a00 / R:0x7f957e9c2ac8>
[29084.580028, 478.880845, 295.635624, 434022.577617, 43578.903110, 123661.592716]
  '': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x157747af0 / R:0x7f957e9c25f8>
[9.531435, 68.646378, 0.286816, 527.425577, 0.321552, NA_real_]
  '.': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x157747cd0 / R:0x7f957e9c24a8>
[-4224.971867, -23516.296288, -9100.800304, -359981.927132, -5801.206401, -19843.862241]
...
  '3': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x157747e60 / R:0x7f957eb2f568>
[-70.571991, -106.295417, -2.332958, -2837.497092, -9.331834, -22.782797]
  ':': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x15a32ab40 / R:0x7f957eb2f418>
[15941.572583, 262.479766, 162.041080, 237892.464570, 23886.067681, 67780.255178]
   ';': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x1f1467a50 / R:0x7f957eb2f108>
[1769.724984, 1769.724984, 1769.724984, 1769.724984, 1769.724984, 1769.724984]
  '?': <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x15b402410 / R:0x7f957eb2efb8>
[1786.000000, 1750.000000, 1771.000000, 1794.000000, 1893.000000, 1852.000000]

In [92]:
train_plot = ggplot.ggplot(R_train_det)

train_details_plot = train_plot + ggplot.aes_string(x='YEAR') + ggplot.geom_bar()
    
train_plot.plot()

AttributeError: 'ListVector' object has no attribute 'plot'